In [151]:
# using Pkg
# Pkg.add("SpecialFunctions")
using Distributions
using Gadfly
using SpecialFunctions
using Cairo

In [152]:
d = Beta(2, 2)
sample = rand(d, 1000)

1000-element Vector{Float64}:
 0.8545998963343767
 0.3146366778444569
 0.25190689902203317
 0.13067334999963304
 0.3529872985858248
 0.20806475348886705
 0.8427068396389958
 0.4128566567403437
 0.05578097426451373
 0.44969982653348467
 ⋮
 0.7177083372180927
 0.8719333553533368
 0.4300812625765916
 0.8231725385273645
 0.35224040323661715
 0.8768135700024954
 0.6462570059613765
 0.6126968459101958
 0.7972508553908163

In [153]:
function dL_da(sample, a, b)
    n = length(sample);
    return n * (digamma(a) - digamma(a + b)) - sum(log.(sample));
end

function dL_db(sample, a, b)
    n = length(sample);
    return n * (digamma(b) - digamma(a + b)) - sum(log.(1 .- sample));
end

dL_db (generic function with 1 method)

In [154]:
function grad_descent_beta(sample)
    n = length(sample);
    max_iter = 1000;
    epsilon = 1e-3;
    lr = 1e-3;

    # Initialize parameters
    alpha = rand() * 10;
    beta = rand() * 10;

    for i = 1:max_iter
        alpha_new = alpha - lr * dL_da(sample, alpha, beta);
        beta_new = beta - lr * dL_db(sample, alpha, beta);
        # Ensure parameters remain positive
        alpha_new = max(alpha_new, 1e-3);
        beta_new = max(beta_new, 1e-3);

        if abs(alpha_new - alpha) < epsilon && abs(beta_new - beta) < epsilon
            alpha = alpha_new;
            beta = beta_new;
            # println("Converged in iteration $i");
            # println("Estimated parameters: alpha = $alpha, beta = $beta");
            break;
        end
        alpha = alpha_new;
        beta = beta_new;
    end
    return alpha, beta;
end

grad_descent_beta (generic function with 1 method)

In [155]:
grad_descent_beta(sample)

(2.111670869420904, 2.1709783392577395)

In [156]:
sample_sizes = collect(2:1000)
est_alpha = zeros(length(sample_sizes))
est_beta = zeros(length(sample_sizes))

for i = 1:length(sample_sizes)
    sample = rand(d, sample_sizes[i])
    a, b = grad_descent_beta(sample)
    est_alpha[i] = a
    est_beta[i] = b
end

myplot1 = plot(x=sample_sizes, y=est_alpha, Geom.line,
    Guide.xlabel("Sample Size"),
    Guide.ylabel("Estimated Alpha"),
    Guide.title("Consistency of Alpha"),
    Theme(background_color="white")
)

myplot2 = plot(x=sample_sizes, y=est_beta, Geom.line,
    Guide.xlabel("Sample Size"),
    Guide.ylabel("Estimated Beta"),
    Guide.title("Consistency of Beta"),
    Theme(background_color="white")
)

final_plot = hstack(myplot1, myplot2)
draw(PNG("estimation.png", 10inch, 5inch), final_plot)
